# Function Decorators   <a href="https://colab.research.google.com/github/Ahmad-Zaki/Python-Notes/blob/main/Function%20Decorators/function-decorators.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

## Introduction

In Python, a decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

If you didn't know: everything in python, including functions, are objects. Functions have attributes, can be assigned to variables, and even passed as arguments to (or returned from) other functions! This behavior is what allows us to use decorators in Python to add new functionality to an existing function without modifying its structure.

Before jumping straight to decorators, it is essential to understand this concept in order to understand how decorators work. Therefore, we'll go through some examples first and then start looking into decorators.

## Functions as objects

Functions and methods are objects of type **`callable`** as they can be called. In fact, any object which implements the magic method (`__call__`) is a callable object.

In the following sections, we will see the properties of callable objects and how it is utilized by decorators.

### Assigning Functions to Variables

In the following examples, we will see how a function can be assigned to a variable, and use this variable to call the function. 

In [1]:
my_print_function = print

my_print_function("This is my own printing function!")

This is my own printing function!


In [2]:
def plus_one(number = 10):
    """Takes a number, add 1 to it, then returns it.
    
    Parameters
    ----------
    number:
        the number to which a one will be added.
        
    Returns
    -------
    number + 1
    """
    return number + 1

add_one = plus_one
add_one(5)

6

A function carry many attributes, like its name, docs, default arguments, ...etc. All these attributes can be accessed like this:

In [3]:
print(plus_one.__name__)
print(add_one.__doc__)
print(add_one.__defaults__)

plus_one
Takes a number, add 1 to it, then returns it.
    
    Parameters
    ----------
    number:
        the number to which a one will be added.
        
    Returns
    -------
    number + 1
    
(10,)


### Passing functions as arguments

Functions can be passed to other functions just like any other object. This allows us to create more dynamic functions without much effort. 

In [4]:
def plus_one(num):
    return num + 1

def minus_one(num):
    return num - 1

def operation(num, func):
    return func(num)

print(operation(42, plus_one))
print(operation(42, minus_one))     

43
41


## Nested Functions

In python, a Nested function is and function that is contained inside another function. This is done simply by defining a function directly inside the body of another function:

In [5]:
def operation(num):
    def plus_one():
        return num + 1
    result = plus_one()
    return result

operation(11)

12

Have you noticed that even though we didn't define the variable `num` inside `plus_one`, it still worked just fine? this is because nested functions have access to the scope of their enclosing function (also known as the *non-local scope*). This behavior is critical in decorators, as we'll see later. 

We can also return a nested function instead of directly using it in the enclosing function:

In [6]:
def foo():
    a = 5

    def hi():
        print("hello! " * a)

    return hi

func = foo()
func()

hello! hello! hello! hello! hello! 


Notice that the execution of `hi` depends on the non-local variable `a`. but since `foo` has already completed its execution before calling `hi`, `a` is no longer available in memory! then how does it work?

This is where **Closures** come to the rescue!

## Function Closure

A Closure is a function object that remembers values in enclosing scopes even if they are not present in memory. we can access the closure of `hi` to see what it contains:

In [7]:
print(type(func.__closure__))
print(len(func.__closure__))
print(func.__closure__[0])
print(func.__closure__[0].cell_contents)

<class 'tuple'>
1
<cell at 0x000002B970940A00: int object at 0x000002B96ACA69B0>
5


We can see that the closure of `hi` is a tuple that contains one cell. This cell has the value of the non-local variable `a`, which `hi` needs in order to work properly.

## Decorators

Now that we know how functions operate in Python and how to deal with functions as objects, we can go ahead and explore *Decorators*. A Decorator is simply a function that wraps other functions to modify their inputs, outputs, or functionalities.

Consider the simple function `say_hi`, which returns "Welcome to my notebook!"

In [8]:
def say_hi():
    return "Welcome to my notebook!"

say_hi()

'Welcome to my notebook!'

What if we want this function to return an upper-cased string instead? In this simple example, we can directly change its output, but decorators provide another way to do it without touching the original function at all!

In [9]:
def uppercase(function: callable):

    def wrapper():
        output = function()
        make_uppercase = output.upper()
        return make_uppercase

    return wrapper

Our `uppercase` decorator takes a function as an argument, this is the function we want to modify. We can see a nested function called `wrapper` inside the decorator. This is where the magic happens: it calls the function we passed to the decorator, executes it to get its output, and modifies the output before returning it. Finally, the decorator returns the `wrapper` function, and this is the function that we'll work with.

Notice that decorators cover all the concepts we went through in the previous sections: it takes a function as an argument, creates a nested function, and returns a function. This is why it was important to understand them well in order to comprehend how decorators work.

Let's try our decorator on `say_hi` to see if it works:

In [10]:
hi_function = uppercase(say_hi)

hi_function()

'WELCOME TO MY NOTEBOOK!'

Remember that `hi_function` is actually the `wrapper` function that the decorator created and returned. Since `wrapper` needs `say_hi` in order to work properly, we can find `say_hi` stored in its closure:

In [11]:
hi_function.__name__

'wrapper'

In [12]:
hi_function.__closure__[0].cell_contents

<function __main__.say_hi()>

Another way to apply a decorator to a function is to *decorate* that function definition with a decorator:

In [13]:
@uppercase
def say_hi():
    return "Welcome to my notebook!"

say_hi()

'WELCOME TO MY NOTEBOOK!'

It's important to emphasize that once we decorate a function, we won't be working with that function anymore, and instead, we'll be working with the `wrapper` function. This means that we won't be able to access the metadata of the original function (name, docs, defaults, ... etc), which may complicate debugging the code.

In [14]:
#Original Function:
def say_hi():
    """Returns a welcoming senctence"""
    return "Welcome to my notebook!"

print(say_hi.__name__)
print(say_hi.__doc__)

say_hi
Returns a welcoming senctence


In [15]:
#Decorated Function:
@uppercase
def say_hi():
    """Returns a welcoming senctence"""
    return "Welcome to my notebook!"

print(say_hi.__name__)
print(say_hi.__doc__)

wrapper
None


In order to fix this behavior, we could manually change all `wrapper` attributes to be the same as our original function. Luckily, Python `functools` module offers an easier way to fix it:

In [16]:
from functools import wraps

def uppercase(function: callable):

    @wraps(function)
    def wrapper():
        output = function()
        make_uppercase = output.upper()
        return make_uppercase
    
    return wrapper

`wraps` decorates `wrapper` and takes `function` as an argument in order to change `wrapper` metadata to be the same as `function` metadata.

In [17]:
@uppercase
def say_hi():
    """Returns a welcoming senctence"""
    return "Welcome to my notebook!"

print(say_hi.__name__)
print(say_hi.__doc__)

say_hi
Returns a welcoming senctence


It is advisable and good practice to always use `functools.wraps` when defining decorators. It will save you a lot of headaches in debugging.

## Real-world examples

So far we have seen a simple example on decorators. In fact, decorators can be used to add very useful functionalities to functions without adding or changing any line of code in their bodies.

For example, consider the following `timer` decorator, which records and prints how long a function takes to run.

In [18]:
import time
def timer(func: callable):
    """A decorator that prints how long a function took to run."""
    
    # Define the wrapper function to return.
    # Notice that it is defined with the general-purpose *args and **kwargs in order to be compatible with any function.
    @wraps(func)
    def wrapper(*args, **kwargs):
        # When wrapper() is called, get the current time.
        t_start = time.time()

        # Call the decorated function and store the result.
        result = func(*args, **kwargs)

        # Get the total time it took to run, and print it.
        t_total = time.time() - t_start
        
        print(f'{func.__name__} took {t_total}s')
        return result

    return wrapper

In [19]:
@timer
def sleep_n_seconds(n):
    time.sleep(n)

In [20]:
sleep_n_seconds(5)

sleep_n_seconds took 5.0013885498046875s


In [21]:
sleep_n_seconds(10)

sleep_n_seconds took 10.012671947479248s


Another useful example is the `memoize` decorator, which stores the output of a function for specific arguments. This can be very helpful if your function takes too long to execute and you need to use it with the same arguments multiple times.

In [22]:
def memoize(func):
    """Store the results of the decorated function for fast lookup"""

    # Store results in a dict that maps arguments to results
    cache = {}

    # Define the wrapper function to return.
    @wraps(func)
    def wrapper(*args, **kwargs):
        # If these arguments haven't been seen before, call func() and store the result.
        # kwargs is ignored because dictionaries are not hashable.
        if args not in cache:
            cache[args] = func(*args, **kwargs)

        return cache[args]

    return wrapper

In [23]:
@timer
@memoize
def first_n(n: int):
    num = 0
    nums = []
    while num < n:
        nums.append(num)
        num += 1
    return nums

In [24]:
result = first_n(10_000_000)

first_n took 1.53005051612854s


In [25]:
result = first_n(10_000_000)

first_n took 0.0s


## Decorators that take arguments

Sometimes, it’s useful to pass arguments to your decorators. For instance, a `@do_twice` decorator that executes a function twice, could be extended to be a `@do_n_times(n)` decorator. The number of times to execute the decorated function could then be given as an argument.

We know that decorators are functions that return other functions, but how about making a function that returns a decorator? 

Since decorators are just functions, it is possible to make a **decorator factory**: a function that creates and returns functions that act as a decorator. Let's see how we can do it.

In [26]:
def do_n_times(n):
    """Define and return a decorator"""

    def decorator(func):
        """A Decorator"""

        def wrapper(*args, **kwargs):
            """A Wrapper"""
            for i in range(n):
                func(*args, **kwargs)

        return wrapper
        
    return decorator

Calling `@do_n_times(n)` will return `decorator` that repeats a function n times, which is exactly what we wanted.

In [27]:
@do_n_times(3)
def say_hi():
    print("Welcome to my notebook!")

say_hi()

Welcome to my notebook!
Welcome to my notebook!
Welcome to my notebook!


In [28]:
@do_n_times(5)
def say_hi():
    print("Welcome to my notebook!")

say_hi()

Welcome to my notebook!
Welcome to my notebook!
Welcome to my notebook!
Welcome to my notebook!
Welcome to my notebook!


## Summary

Decorators dynamically alter the functionality of a function, method, or class without having to directly use subclasses or change the source code of the function being decorated. Using decorators in Python also ensures that your code is **DRY** (Don't Repeat Yourself). Decorators have several use cases such as:

- Authorization in Python frameworks such as Flask and Django
- Logging
- Measuring execution time
- Synchronization

To learn more about Python decorators check out <a href="https://realpython.com/primer-on-python-decorators/">Primer on Python Decorators</a>.

For more examples on Python decorators chech out <a href="https://wiki.python.org/moin/PythonDecoratorLibrary">Python's Decorator Library</a>.